Nothing fancy here, just some imports I need to run the code, **pickle** is used to load the bottleneck features.

In [14]:
import pickle
import tensorflow as tf
import numpy as np
import pandas as pd
from keras.layers import Conv2D, MaxPool2D, Dense, Activation, Flatten,Input
from keras.models import Model

Here I define some command line flags, this avoids having to manually open and edit the file if I want to change the files we train and validate our model with.

I added a couple of command-line flags to set the number of epochs and batch size. This is more for convenience than anything else.

In [ ]:
flags = tf.app.flags
FLAGS = flags.FLAGS

# command line flags
flags.DEFINE_string('training_file', '', "Bottleneck features training file (.p)")
flags.DEFINE_string('validation_file', '', "Bottleneck features validation file (.p)")
flags.DEFINE_integer('epochs', 50, "The number of epochs.")
flags.DEFINE_integer('batch_size', 256, "The batch size.")


A utility function that loads the bottleneck features from the pickled training and validation files.

In [11]:
def load_bottleneck_data(training_file, validation_file):
    """
    Utility function to load bottleneck features.

    Arguments:
        training_file - String
        validation_file - String
    """
    print("Training file", training_file)
    print("Validation file", validation_file)

    with open(training_file, 'rb') as f:
        train_data = pickle.load(f)
    with open(validation_file, 'rb') as f:
        validation_data = pickle.load(f)

    X_train = train_data['features']
    y_train = train_data['labels']
    X_val = validation_data['features']
    y_val = validation_data['labels']

    return X_train, y_train, X_val, y_val

This is where I defined ( a very simple model, a linear layer (Dense in Keras terms) followed by a softmax activation. The Adam optimizer is used ) and trained the model. Notice FLAGS.training_file and FLAGS.validation_file are passed into load_bottleneck_data. These refer to the command line flags defined earlier.

Here I finded the number of classes for the dataset. **np.unique** returns all the unique elements of a numpy array. The elements of y_train are integers, 0-9 for Cifar10 and  So, when combined with len we get back the number of classes.

In [16]:
def main(_):

    # load bottleneck data of VGG architecture 

    # training_file = "vgg_cifar10_bottleneck_features_train.p"
    # validation_file = "vgg_cifar10_bottleneck_features_validation.p"
    # X_train, y_train, X_val, y_val = load_bottleneck_data(training_file,validation_file)

    # Using Flags
    X_train, y_train, X_val, y_val = load_bottleneck_data(FLAGS.training_file, FLAGS.validation_file)

    print(X_train.shape, y_train.shape)
    print(X_val.shape, y_val.shape)

    
    n_classes = len(np.unique(y_train))

    # define model
    shape_in = X_train.shape[1:]
    inputs = tf.keras.Input(shape=shape_in)
    x = Flatten()(inputs)
    outputs = tf.keras.layers.Dense(n_classes, activation='softmax')(x)
    model = tf.keras.Model(inputs=inputs, outputs=outputs)

    model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

    # Train The Model 
    # model.fit(X_train, y_train, epochs=50, batch_size=256 , validation_data=(X_val, y_val), shuffle=True)

    # using Flags
    model.fit(X_train, y_train, epochs=FLAGS.epochs, batch_size=FLAGS.batch_size, validation_data=(X_val, y_val), shuffle=True)


Training file vgg_cifar10_bottleneck_features_train.p
Validation file vgg_cifar10_bottleneck_features_validation.p
(40000, 1, 1, 512) (40000, 1)
(10000, 1, 1, 512) (10000, 1)
Epoch 1/50
157/157 [==============================] - 1s 5ms/step - loss: 1.3891 - accuracy: 0.5788 - val_loss: 0.7252 - val_accuracy: 0.7554
Epoch 2/50
157/157 [==============================] - 1s 5ms/step - loss: 0.6357 - accuracy: 0.7849 - val_loss: 0.5551 - val_accuracy: 0.8104
Epoch 3/50
157/157 [==============================] - 1s 5ms/step - loss: 0.5197 - accuracy: 0.8245 - val_loss: 0.4931 - val_accuracy: 0.8315
Epoch 4/50
157/157 [==============================] - 1s 5ms/step - loss: 0.4665 - accuracy: 0.8402 - val_loss: 0.4641 - val_accuracy: 0.8396
Epoch 5/50
157/157 [==============================] - 1s 5ms/step - loss: 0.4342 - accuracy: 0.8503 - val_loss: 0.4417 - val_accuracy: 0.8482
Epoch 6/50
157/157 [==============================] - 1s 5ms/step - loss: 0.4123 - accuracy: 0.8577 - val_loss: 0.4

In [ ]:
# parses flags and calls the `main` function above
if __name__ == '__main__':
    tf.app.run()
